In [1]:
import sklearn
import optuna
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import joblib

In [2]:
dic = joblib.load('../../data.joblib')

print(dic.keys())
X = dic['train_features']
Y = dic['train_labels']

dict_keys(['train_features', 'test_features', 'train_labels', 'test_labels', 'train_weights', 'test_weights', 'train_dates'])


In [3]:
print(X.columns)
print(X.shape)
print(X.info())

X.tail()

Index(['%-AVAX/USDTadx-period_3_5m', '%-AVAX/USDTminus_di-period_3_5m',
       '%-AVAX/USDTminus_dm-period_3_5m', '%-AVAX/USDTplus_di-period_3_5m',
       '%-AVAX/USDTplus_dm-period_3_5m', '%-AVAX/USDTcci-period_3_5m',
       '%-AVAX/USDTrsi-period_3_5m', '%-AVAX/USDTmin-period_3_5m',
       '%-AVAX/USDTmax-period_3_5m', '%-AVAX/USDTdema-period_3_5m',
       ...
       '%-AVAX/USDTha_open_shift-25_5m', '%-AVAX/USDTha_close_shift-25_5m',
       '%-AVAX/USDTha_high_shift-25_5m', '%-AVAX/USDTha_low_shift-25_5m',
       '%-AVAX/USDTpct-change_shift-25_5m',
       '%-AVAX/USDTraw_volume_shift-25_5m', '%-AVAX/USDTraw_close_shift-25_5m',
       '%-AVAX/USDTraw_open_shift-25_5m', '%-AVAX/USDTraw_high_shift-25_5m',
       '%-AVAX/USDTraw_low_shift-25_5m'],
      dtype='object', length=15470)
(3637, 15470)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3637 entries, 623 to 4519
Columns: 15470 entries, %-AVAX/USDTadx-period_3_5m to %-AVAX/USDTraw_low_shift-25_5m
dtypes: float64(15470)
memory u

,%-AVAX/USDTadx-period_3_5m,%-AVAX/USDTminus_di-period_3_5m,%-AVAX/USDTminus_dm-period_3_5m,%-AVAX/USDTplus_di-period_3_5m,%-AVAX/USDTplus_dm-period_3_5m,%-AVAX/USDTcci-period_3_5m,%-AVAX/USDTrsi-period_3_5m,%-AVAX/USDTmin-period_3_5m,%-AVAX/USDTmax-period_3_5m,%-AVAX/USDTdema-period_3_5m,...,%-AVAX/USDTha_open_shift-25_5m,%-AVAX/USDTha_close_shift-25_5m,%-AVAX/USDTha_high_shift-25_5m,%-AVAX/USDTha_low_shift-25_5m,%-AVAX/USDTpct-change_shift-25_5m,%-AVAX/USDTraw_volume_shift-25_5m,%-AVAX/USDTraw_close_shift-25_5m,%-AVAX/USDTraw_open_shift-25_5m,%-AVAX/USDTraw_high_shift-25_5m,%-AVAX/USDTraw_low_shift-25_5m
4515,0.181881,-0.712021,-0.882304,-0.667962,-0.836190,-0.064935,0.093349,0.868787,0.850980,0.844870,...,0.697371,0.671085,0.646417,0.707008,0.031853,-0.979217,0.659491,0.659491,0.647969,0.696252
4516,0.194511,-0.514278,-0.823869,-0.750777,-0.890893,-0.714286,-0.202028,0.864811,0.843137,0.838917,...,0.693369,0.666176,0.640598,0.699068,0.065437,-0.980450,0.659491,0.659491,0.636364,0.688363
4517,0.216508,-0.664771,-0.882669,-0.440917,-0.763789,0.500000,0.143990,0.864811,0.843137,0.842803,...,0.688871,0.670103,0.642532,0.703038,0.099040,-0.984762,0.663405,0.659491,0.644101,0.692308
4518,0.177192,0.061867,-0.579720,-0.670673,-0.842626,-1.000000,-0.309290,0.860835,0.843137,0.835457,...,0.688620,0.668139,0.642532,0.699068,-0.035317,-0.983362,0.651663,0.667319,0.644101,0.688363
4519,0.140684,-0.282180,-0.719903,-0.777546,-0.895184,-0.766234,-0.657990,0.840954,0.843137,0.818332,...,0.687495,0.657339,0.634883,0.683189,0.065437,-0.952914,0.651663,0.647750,0.636364,0.672584


In [4]:
features = ["%-AVAX/USDTmom-period_14_shift-1_5m", "%-AVAX/USDTroc-period_14_5m", "%-AVAX/USDTroc-period_14_shift-1_5m", "%-AVAX/USDTmom-period_14_5m", "%-AVAX/USDTwillr-period_21_5m", "%-AVAX/USDTcci-period_21_5m", "%-AVAX/USDTcci-period_20_5m", "%-AVAX/USDTrsi-period_14_5m", "%-AVAX/USDTwbb_percent_24_5m", "%-AVAX/USDTwillr-period_24_5m", "%-AVAX/USDTcci-period_24_5m", "%-AVAX/USDTrsi-period_9_5m", "%-AVAX/USDTwbb_percent_21_5m", "%-AVAX/USDTlinearreg_angle-period_20_5m", "%-AVAX/USDTcci-period_24_shift-1_5m", "%-AVAX/USDTwillr-period_20_5m", "%-AVAX/USDTrsi-period_12_5m", "%-AVAX/USDTlinearreg_slope-period_20_5m", "%-AVAX/USDTlinearreg_slope-period_21_5m", "%-AVAX/USDTcci-period_20_shift-1_5m", "%-AVAX/USDTmom-period_12_shift-2_5m", "%-AVAX/USDTrsi-period_10_5m", "%-AVAX/USDTwbb_percent_20_5m", "%-AVAX/USDTrsi-period_10_shift-1_5m", "%-AVAX/USDTcci-period_14_5m", "%-AVAX/USDTmom-period_12_shift-3_5m", "%-AVAX/USDTwbb_percent_14_5m", "%-AVAX/USDTlinearreg_slope-period_14_shift-1_5m", "%-AVAX/USDTcci-period_21_shift-1_5m", "%-AVAX/USDTroc-period_12_5m", "%-AVAX/USDTlinearreg_angle-period_14_shift-2_5m", "%-AVAX/USDTmom-period_14_shift-2_5m", "%-AVAX/USDTwillr-period_20_shift-1_5m", "%-AVAX/USDTroc-period_14_shift-2_5m", "%-AVAX/USDTwbb_percent_12_5m", "%-AVAX/USDTtrix-period_5_5m", "%-AVAX/USDTlinearreg_angle-period_21_5m", "%-AVAX/USDTmom-period_12_5m", "%-AVAX/USDTaroonosc_5m", "%-AVAX/USDTmfi-period_14_5m", "%-AVAX/USDTroc-period_12_shift-3_5m", "%-AVAX/USDTmom-period_20_5m", "%-AVAX/USDTlinearreg_angle-period_14_shift-3_5m", "%-AVAX/USDTcci-period_20_shift-2_5m", "%-AVAX/USDTlinearreg_slope-period_14_shift-2_5m", "%-AVAX/USDTwillr-period_21_shift-1_5m", "%-AVAX/USDTroc-period_12_shift-1_5m", "%-AVAX/USDTroc-period_9_shift-6_5m", "%-AVAX/USDTrsi-period_14_shift-1_5m", "%-AVAX/USDTmom-period_12_shift-1_5m", "%-AVAX/USDTlinearreg_angle-period_14_5m", "%-AVAX/USDTlinearreg_slope-period_12_shift-4_5m", "%-AVAX/USDTroc-period_20_5m", "%-AVAX/USDTmom-period_10_shift-5_5m", "%-AVAX/USDTclose-bb_lower-period_20_5m", "%-AVAX/USDTwillr-period_14_5m", "%-AVAX/USDTwbb_percent_10_5m", "%-AVAX/USDTrsi-period_24_5m", "%-AVAX/USDTlinearreg_angle-period_24_5m", "%-AVAX/USDTwbb_percent_21_shift-2_5m", "%-AVAX/USDTrsi-period_12_shift-1_5m", "%-AVAX/USDTwillr-period_24_shift-1_5m", "%-AVAX/USDTlinearreg_angle-period_12_shift-4_5m", "%-AVAX/USDTtrix-period_5_shift-1_5m", "%-AVAX/USDTclose-bb_lower-period_21_5m", "%-AVAX/USDTroc-period_10_shift-5_5m", "%-AVAX/USDTwbb_percent_20_shift-1_5m", "%-AVAX/USDTlinearreg_angle-period_14_shift-1_5m", "%-AVAX/USDTwbb_percent_24_shift-1_5m", "%-AVAX/USDTcci-period_21_shift-2_5m", "%-AVAX/USDTwbb_percent_21_shift-1_5m", "%-AVAX/USDTplus_di-period_10_5m", "%-AVAX/USDTtrix-period_5_shift-2_5m", "%-AVAX/USDTcci-period_14_shift-1_5m", "%-AVAX/USDTlinearreg_slope-period_14_shift-3_5m", "%-AVAX/USDTlinearreg_slope-period_14_5m", "%-AVAX/USDTlinearreg_slope-period_20_shift-1_5m", "%-AVAX/USDTaroonosc_shift-1_5m", "%-AVAX/USDTlinearreg_angle-period_20_shift-2_5m", "%-AVAX/USDTmfi-period_12_5m", "%-AVAX/USDTwillr-period_14_shift-2_5m", "%-AVAX/USDTroc-period_9_5m", "%-AVAX/USDTlinearreg_slope-period_12_shift-5_5m", "%-AVAX/USDTcci-period_50_5m", "%-AVAX/USDTroc-period_21_5m", "%-AVAX/USDTaroonosc_shift-2_5m", "%-AVAX/USDTaroonup_5m", "%-AVAX/USDTlinearreg_angle-period_12_shift-5_5m", "%-AVAX/USDTmom-period_9_shift-6_5m", "%-AVAX/USDTlinearreg_angle-period_20_shift-1_5m", "%-AVAX/USDTwillr-period_14_shift-1_5m", "%-AVAX/USDTwbb_percent_50_5m", "%-AVAX/USDTclose-bb_lower-period_24_5m", "%-AVAX/USDTlinearreg_slope-period_12_shift-3_5m", "%-AVAX/USDTroc-period_12_shift-2_5m", "%-AVAX/USDTwbb_percent_9_5m", "%-AVAX/USDTrsi-period_21_5m", "%-AVAX/USDTmfi-period_14_shift-1_5m", "%-AVAX/USDTwillr-period_12_5m", "%-AVAX/USDTaroonup_shift-1_5m", "%-AVAX/USDTwillr-period_20_shift-2_5m", "%-AVAX/USDTcci-period_12_5m", "%-AVAX/USDTrsi-period_20_5m", "%-AVAX/USDTroc-period_10_shift-4_5m", "%-AVAX/USDTlinearreg_angle-period_14_shift-4_5m", "%-AVAX/USDTwbb_percent_12_shift-1_5m", "%-AVAX/USDTwbb_percent_20_shift-2_5m", "%-AVAX/USDTlinearreg_slope-period_12_5m", "%-AVAX/USDTtrix-period_10_5m", "%-AVAX/USDTlinearreg_slope-period_14_shift-4_5m", "%-AVAX/USDTplus_di-period_10_shift-1_5m", "%-AVAX/USDTrsi-period_5_5m", "%-AVAX/USDTtrix-period_9_5m", "%-AVAX/USDTtrix-period_5_shift-4_5m", "%-AVAX/USDTroc-period_10_shift-1_5m", "%-AVAX/USDTminus_di-period_9_5m", "%-AVAX/USDTlinearreg_slope-period_21_shift-1_5m", "%-AVAX/USDTmacdhist_5m", "%-AVAX/USDTwillr-period_20_shift-3_5m", "%-AVAX/USDTlinearreg_angle-period_12_shift-6_5m", "%-AVAX/USDTlinearreg_angle-period_12_5m", "%-AVAX/USDTrsi-period_9_shift-1_5m", "%-AVAX/USDTmom-period_10_shift-3_5m", "%-AVAX/USDTwbb_percent_14_shift-1_5m", "%-AVAX/USDTlinearreg_slope-period_20_shift-2_5m", "%-AVAX/USDTtrix-period_3_5m", "%-AVAX/USDTrsi-period_10_shift-3_5m", "%-AVAX/USDTplus_di-period_9_5m", "%-AVAX/USDTclose-bb_lower-period_14_5m", "%-AVAX/USDTlinearreg_angle-period_12_shift-2_5m",
    "%-AVAX/USDTcci-period_14_shift-4_5m", "%-AVAX/USDTwbb_percent_55_5m", "%-AVAX/USDTmom-period_9_5m", "%-AVAX/USDTroc-period_9_shift-1_5m", "%-AVAX/USDTplus_di-period_12_5m", "%-AVAX/USDTcci-period_55_5m", "%-AVAX/USDTmom-period_10_5m", "%-AVAX/USDTwbb_percent_50_shift-1_5m", "%-AVAX/USDTminus_di-period_10_5m", "%-AVAX/USDTlinearreg_slope-period_24_5m", "%-AVAX/USDTrsi-period_10_shift-2_5m", "%-AVAX/USDTroc-period_14_shift-3_5m", "%-AVAX/USDTcci-period_24_shift-2_5m", "%-AVAX/USDTcci-period_10_5m", "%-AVAX/USDTtrix-period_5_shift-3_5m", "%-AVAX/USDTlinearreg_angle-period_10_shift-6_5m", "%-AVAX/USDTwbb_percent_14_shift-2_5m", "%-AVAX/USDTroc-period_10_5m", "%-AVAX/USDTlinearreg_slope-period_9_shift-7_5m", "%-AVAX/USDTclose-bb_lower-period_21_shift-1_5m", "%-AVAX/USDTlinearreg_slope-period_10_shift-6_5m", "%-AVAX/USDTclose-bb_lower-period_20_shift-1_5m", "%-AVAX/USDTmom-period_21_5m", "%-AVAX/USDTrsi-period_14_shift-2_5m", "%-AVAX/USDTplus_di-period_5_5m", "%-AVAX/USDTcci-period_20_shift-3_5m", "%-AVAX/USDTmom-period_20_shift-1_5m", "%-AVAX/USDTrsi-period_9_shift-2_5m", "%-AVAX/USDTroc-period_10_shift-3_5m", "%-AVAX/USDTroc-period_24_5m", "%-AVAX/USDTroc-period_5_shift-10_5m", "%-AVAX/USDTaroonup_shift-2_5m", "%-AVAX/USDTwillr-period_21_shift-2_5m", "%-AVAX/USDTao_5m", "%-AVAX/USDTcci-period_14_shift-2_5m", "%-AVAX/USDTroc-period_9_shift-5_5m", "%-AVAX/USDTwillr-period_24_shift-2_5m", "%-AVAX/USDTcci-period_24_shift-3_5m", "%-AVAX/USDTcci-period_9_5m", "%-AVAX/USDTplus_di-period_9_shift-2_5m", "%-AVAX/USDTlinearreg_angle-period_12_shift-3_5m", "%-AVAX/USDTminus_di-period_12_5m", "%-AVAX/USDTwillr-period_10_5m", "%-AVAX/USDTroc-period_10_shift-2_5m", "%-AVAX/USDTroc-period_10_shift-6_5m", "%-AVAX/USDTlinearreg_angle-period_9_shift-8_5m", "%-AVAX/USDTcci-period_14_shift-5_5m", "%-AVAX/USDTrsi-period_3_5m", "%-AVAX/USDTwillr-period_12_shift-1_5m", "%-AVAX/USDTwbb_percent_12_shift-2_5m", "%-AVAX/USDTmfi-period_12_shift-3_5m", "%-AVAX/USDTtrix-period_9_shift-2_5m", "%-AVAX/USDTrsi-period_12_shift-2_5m", "%-AVAX/USDTcci-period_14_shift-3_5m", "%-AVAX/USDTaroondown_shift-1_5m", "%-AVAX/USDTwillr-period_50_5m", "%-AVAX/USDTmom-period_24_5m", "%-AVAX/USDTlinearreg_angle-period_14_shift-5_5m", "%-AVAX/USDTlinearreg_slope-period_9_shift-8_5m", "%-AVAX/USDTwillr-period_14_shift-3_5m", "%-AVAX/USDTmfi-period_12_shift-2_5m", "%-AVAX/USDTcci-period_12_shift-3_5m", "%-AVAX/USDTmom-period_12_shift-4_5m", "%-AVAX/USDTrsi-period_24_shift-1_5m", "%-AVAX/USDTaroondown_5m", "%-AVAX/USDTrsi-period_12_shift-3_5m", "%-AVAX/USDTlinearreg_slope-period_12_shift-2_5m", "%-AVAX/USDTwbb_percent_10_shift-1_5m", "%-AVAX/USDTlinearreg_angle-period_10_5m", "%-AVAX/USDTmacdhist_shift-4_5m", "%-AVAX/USDTcci-period_14_shift-6_5m", "%-AVAX/USDTlinearreg_angle-period_10_shift-7_5m", "%-AVAX/USDTrsi-period_5_shift-1_5m", "%-AVAX/USDTclose-bb_lower-period_50_5m", "%-AVAX/USDTclose-bb_lower-period_24_shift-1_5m", "%-AVAX/USDTtrix-period_9_shift-1_5m", "%-AVAX/USDTwillr-period_9_5m", "%-AVAX/USDTlinearreg_slope-period_10_shift-5_5m", "%-AVAX/USDTlinearreg_angle-period_21_shift-1_5m", "%-AVAX/USDTwbb_percent_5_5m", "%-AVAX/USDTclose-bb_lower-period_12_5m", "%-AVAX/USDTcci-period_21_shift-3_5m", "%-AVAX/USDTmacdhist_shift-1_5m", "%-AVAX/USDTmfi-period_10_5m", "%-AVAX/USDTlinearreg_slope-period_14_shift-5_5m", "%-AVAX/USDTwillr-period_12_shift-6_5m", "%-AVAX/USDTmfi-period_12_shift-1_5m", "%-AVAX/USDTmacd_5m", "%-AVAX/USDTwillr-period_12_shift-2_5m", "%-AVAX/USDTplus_di-period_3_5m", "%-AVAX/USDTmom-period_9_shift-5_5m", "%-AVAX/USDTroc-period_9_shift-7_5m", "%-AVAX/USDTclose-bb_lower-period_10_5m", "%-AVAX/USDTlinearreg_slope-period_12_shift-6_5m", "%-AVAX/USDTlinearreg_slope-period_10_5m", "%-AVAX/USDTmom-period_10_shift-4_5m", "%-AVAX/USDTrsi-period_21_shift-1_5m", "%-AVAX/USDTrsi-period_20_shift-1_5m", "%-AVAX/USDTcci-period_12_shift-2_5m", "%-AVAX/USDTmfi-period_9_shift-5_5m", "%-AVAX/USDTmom-period_10_shift-2_5m", "%-AVAX/USDTtrix-period_5_shift-5_5m", "%-AVAX/USDTwillr-period_14_shift-4_5m", "%-AVAX/USDTlinearreg_angle-period_10_shift-3_5m", "%-AVAX/USDTcci-period_12_shift-4_5m", "%-AVAX/USDTcci-period_12_shift-1_5m", "%-AVAX/USDTlinearreg_slope-period_9_5m", "%-AVAX/USDTmom-period_5_shift-10_5m", "%-AVAX/USDTlinearreg_slope-period_10_shift-4_5m", "%-AVAX/USDTwillr-period_21_shift-3_5m", "%-AVAX/USDTminus_di-period_14_shift-1_5m", "%-AVAX/USDTrsi-period_5_shift-2_5m", "%-AVAX/USDTwbb_percent_10_shift-2_5m", "%-AVAX/USDTcci-period_50_shift-1_5m", "%-AVAX/USDTwbb_percent_14_shift-3_5m", "%-AVAX/USDTroc-period_20_shift-1_5m", "%-AVAX/USDTaroonup_shift-3_5m", "%-AVAX/USDTminus_di-period_14_5m", "%-AVAX/USDTmom-period_5_5m", "%-AVAX/USDTmom-period_3_5m", "%-AVAX/USDTplus_di-period_9_shift-1_5m", "%-AVAX/USDTclose-bb_lower-period_14_shift-1_5m", "%-AVAX/USDTcci-period_12_shift-6_5m", "%-AVAX/USDTmom-period_14_shift-3_5m", "%-AVAX/USDTlinearreg_slope-period_5_shift-11_5m", "%-AVAX/USDTmom-period_10_shift-6_5m", "%-AVAX/USDTroc-period_5_5m",
     "%-AVAX/USDTclose-bb_lower-period_9_5m", "%-AVAX/USDTlinearreg_slope-period_12_shift-1_5m", "%-AVAX/USDTlinearreg_slope-period_10_shift-7_5m", "%-AVAX/USDTminus_di-period_12_shift-1_5m", "%-AVAX/USDTclose-bb_lower-period_21_shift-2_5m", "%-AVAX/USDTmfi-period_10_shift-2_5m", "%-AVAX/USDTrsi-period_20_shift-2_5m", "%-AVAX/USDTroc-period_24_shift-1_5m", "%-AVAX/USDTlinearreg_angle-period_5_shift-11_5m", "%-AVAX/USDTtrix-period_5_shift-6_5m", "%-AVAX/USDTwbb_percent_24_shift-3_5m", "%-AVAX/USDTroc-period_3_shift-12_5m", "%-AVAX/USDTminus_di-period_9_shift-1_5m", "%-AVAX/USDTmom-period_9_shift-3_5m", "%-AVAX/USDTwbb_percent_55_shift-1_5m", "%-AVAX/USDTlinearreg_angle-period_10_shift-8_5m", "%-AVAX/USDTlinearreg_angle-period_9_shift-7_5m", "%-AVAX/USDTclose-bb_lower-period_5_5m", "%-AVAX/USDTmfi-period_10_shift-4_5m", "%-AVAX/USDTmacdhist_shift-3_5m", "%-AVAX/USDTmom-period_3_shift-12_5m", "%-AVAX/USDTaroonosc_shift-3_5m", "%-AVAX/USDTmfi-period_10_shift-5_5m", "%-AVAX/USDTroc-period_12_shift-4_5m", "%-AVAX/USDTwbb_percent_100_5m", "%-AVAX/USDTlinearreg_angle-period_5_5m", "%-AVAX/USDTcci-period_5_5m", "%-AVAX/USDTslowd_shift-9_5m", "%-AVAX/USDTmfi-period_9_shift-3_5m", "%-AVAX/USDTroc-period_3_5m", "%-AVAX/USDTcci-period_14_shift-7_5m", "%-AVAX/USDTwillr-period_12_shift-4_5m", "%-AVAX/USDTlinearreg_angle-period_12_shift-1_5m", "%-AVAX/USDTminus_di-period_5_5m", "%-AVAX/USDTmom-period_9_shift-7_5m", "%-AVAX/USDTwbb_percent_14_shift-4_5m", "%-AVAX/USDTwbb_percent_24_shift-2_5m", "%-AVAX/USDTcci-period_24_shift-4_5m", "%-AVAX/USDTwbb_percent_20_shift-3_5m", "%-AVAX/USDTclose-bb_lower-period_55_5m", "%-AVAX/USDTwillr-period_12_shift-3_5m", "%-AVAX/USDTlinearreg_angle-period_20_shift-3_5m", "%-AVAX/USDTmom-period_10_shift-1_5m"
    ]

X = X[features]

In [5]:
print(Y.columns)
print(Y.shape)
print(Y.info())
print(Y.value_counts())

Y.tail()

Index(['&s-up_down_same'], dtype='object')
(3637, 1)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3637 entries, 623 to 4519
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   &s-up_down_same  3637 non-null   object
dtypes: object(1)
memory usage: 56.8+ KB
None
&s-up_down_same
up                 1796
down               1775
upper                66
dtype: int64


,&s-up_down_same
4515,up
4516,up
4517,up
4518,up
4519,up


In [6]:
Y['&s-up_down_same'] = Y['&s-up_down_same'].replace({"upper": 0, "up": 1, "down": 2})
Y.tail()

,&s-up_down_same
4515,1
4516,1
4517,1
4518,1
4519,1


In [7]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y,
                                            test_size = 0.30,
                                            shuffle=False,
                                        )

print(X_train.shape, ' ',X_test.shape, " ", y_train.shape, ' ',y_test.shape)

(2545, 300)   (1092, 300)   (2545, 1)   (1092, 1)


In [8]:
from lightgbm import LGBMClassifier

def light_gbm_classifier_optuna(trial):
    model = LGBMClassifier(
        boosting_type=trial.suggest_categorical("boosting_type", ["gbdt", "dart", "rf"]),
        num_leaves=trial.suggest_int('num_leaves', 21, 70),
        learning_rate=trial.suggest_float("learning_rate", 0.1, 0.9),
        n_estimators=trial.suggest_categorical("n_estimators", [1000, 1500, 5000]),
        class_weight=trial.suggest_categorical("class_weight", ["balanced", None]),
    )    
    cv = sklearn.model_selection.cross_val_score(model, X_train, y_train['&s-up_down_same'].ravel(), scoring="f1_micro", cv = 3)
    f1_score = cv.mean()
    return f1_score
    

# Step 4: Running it
study = optuna.create_study(direction="maximize")
study.optimize(light_gbm_classifier_optuna, n_trials=500)

[I 2023-01-15 13:13:11,553] A new study created in memory with name: no-name-7b1513ea-b34f-44a8-b432-4bcfe8a5d568
[I 2023-01-15 13:13:46,270] Trial 0 finished with value: 0.8455762791593514 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 44, 'learning_rate': 0.5985680085140623, 'n_estimators': 1500, 'class_weight': None}. Best is trial 0 with value: 0.8455762791593514.
[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .

/home/hamza/anaconda3/lib/python3.9

In [11]:
print(study.best_params)
study.best_value

{'boosting_type': 'gbdt', 'num_leaves': 33, 'learning_rate': 0.15898968724390175, 'n_estimators': 5000, 'class_weight': None}


0.9634568230474624

In [9]:
study.best_params

{'boosting_type': 'dart',
 'num_leaves': 51,
 'learning_rate': 0.09486903727653351,
 'n_estimators': 1000,
 'class_weight': None}

In [16]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(
    boosting_type="dart",
    num_leaves=51,
    learning_rate=0.09486903727653351,
    n_estimators=1000,
    class_weight=None,
    )    

cv = sklearn.model_selection.cross_val_score(model, X_train, y_train['&s-up_down_same'].ravel(), scoring="f1_micro", cv = 3)
f1_score = cv.mean()
f1_score

0.9611011104443999

In [ ]:
def xgboost_classifier_optuna(trial):
    model = 
    
    cv = sklearn.model_selection.cross_val_score(model, x_train, y_train, scoring="f1_micro", cv = 3)
    f1_score = cv.mean()
    return f1_score
    

# Step 4: Running it
study = optuna.create_study(direction="maximize")
study.optimize(, n_trials=20)

In [ ]:
def xgbrfoost_classifier_optuna(trial):
    model = 
    
    cv = sklearn.model_selection.cross_val_score(model, x_train, y_train, scoring="f1_micro", cv = 3)
    f1_score = cv.mean()
    return f1_score
    

# Step 4: Running it
study = optuna.create_study(direction="maximize")
study.optimize(, n_trials=20)